In [1]:
#Welcome to my notebook of Segmenting and Clustering Neighborhoods in Toronto
# Depending where you want to run this notebook you may requiere some libraries
# such as beautifulsoup4.
#!conda install -c conda-forge beautifulsoup4 --yes #uncomment if install needed

In [21]:
from bs4 import BeautifulSoup #library for web scraping
import requests


##  Creating the DataFrame by using BeautifulSoup  

In [22]:
# First I need  obtain the data from Wikipedia
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [23]:
source= requests.get(url).text

In [24]:
soup= BeautifulSoup(source,'lxml')

In [25]:
table=soup.find('table')

In [26]:
import pandas as pd

In [27]:
tables = pd.read_html(source)

In [28]:
#converting in dataframe
df = pd.DataFrame(tables[0])
df.columns=['Postcode','Borough','Neighbourhood']


## Cleaning Data

In [29]:
# We need remove the not assigend from boroughs and grouping them with common postcode
dfa=df[df.Borough != 'Not assigned']
dfb=dfa.groupby("Postcode").agg(lambda x:','.join(set(x)))
dfb=dfb.reset_index()
dfb.head(10)


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Port Union,Rouge Hill"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Ionview,East Birchmount Park,Kennedy Park"
7,M1L,Scarborough,"Golden Mile,Oakridge,Clairlea"
8,M1M,Scarborough,"Scarborough Village West,Cliffcrest,Cliffside"
9,M1N,Scarborough,"Cliffside West,Birch Cliff"


In [30]:
dfb.shape

(103, 3)

In [31]:
#conda install -c conda-forge geocoder

In [47]:
#import geocoder # import geocoder
# doesnt work


In [39]:
co_df = pd.read_csv('https://cocl.us/Geospatial_data')
print(co_df.head())
print('shape:',co_df.shape)   # so we have the data of the latitute and longitude

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476
shape: (103, 3)


In [41]:
co_df.rename(columns={'Postal Code':'Postcode'}, inplace=True)
co_df.head()# we use the same notation

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [43]:

#sort both dataframes by Postcode 
co_df.sort_values(by = ['Postcode'], inplace=True)


In [44]:
dfb.sort_values(by = ['Postcode'], inplace=True)

In [48]:
#we just merge the data frames
final_df = dfb.merge(co_df, left_on='Postcode', right_on='Postcode', )
final_df.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Port Union,Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Ionview,East Birchmount Park,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile,Oakridge,Clairlea",43.711112,-79.284577
8,M1M,Scarborough,"Scarborough Village West,Cliffcrest,Cliffside",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West,Birch Cliff",43.692657,-79.264848


In [49]:
final_df.shape

(103, 5)